# **1. Starting Setup**




In [1]:
!pip install -q "transformers>=4.40.0" datasets accelerate bitsandbytes sentencepiece protobuf

!pip install -q transformers accelerate bitsandbytes sentence-transformers networkx

!pip install flash-attn --no-build-isolation

!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 28.4 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 59.7 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for flash-attn: filename=flash_attn-2.8.3-cp312-cp312-linux_x86_64.whl size=256040057 sha256=f25da18657a87fc83dc1bfb8b7751b82246e9db355510226b674fd437c34b5fb
  Stored in directory: /root/.cache/pip/wheels/3d/59/46/f282c12c73dd4bb3c2e3fe199f1a0d0f8cec06df0cccfeee27
Successfully built flash-attn


In [6]:
import torch
import re
from tqdm import tqdm


import torch
import networkx as nx
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from sentence_transformers import SentenceTransformer, util
import json
from pathlib import Path
from typing import Dict, List, Set
# Configurazione del dispositivo (GPU se disponibile)
device = "cuda" if torch.cuda.is_available() else "cpu"

## 1.1 Setting runtime

In [3]:
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
else:
    print("⚠️ No GPU. Go to Runtime → Change runtime type → set GPU.")

CUDA available: True
GPU: Tesla T4


## 1.2 Import Dataset

In [4]:
!git clone https://github.com/sooo66/semeval2026-task12-dataset.git
!ls semeval2026-task12-dataset

Cloning into 'semeval2026-task12-dataset'...
remote: Enumerating objects: 63, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 63 (delta 27), reused 46 (delta 13), pack-reused 0 (from 0)
Receiving objects: 100% (63/63), 6.71 MiB | 10.64 MiB/s, done.
Resolving deltas: 100% (27/27), done.
dev_data   sample_data			   test_data
README.md  semeval2026-task12-dataset-old  train_data


## 1.3 Constants definition

In [14]:

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Root of the cloned repo in Colab
DATA_ROOT = Path("/kaggle/working/semeval2026-task12-dataset")

# Choose the split you want to evaluate: "train_data", "dev_data", or "sample_data"
SPLIT = "dev_data"  # you can change this

QUESTIONS_FILE = DATA_ROOT / SPLIT / "questions.jsonl"
DOCS_FILE      = DATA_ROOT / SPLIT / "docs.json"

print("Questions file:", QUESTIONS_FILE)
print("Docs file:", DOCS_FILE)

Questions file: /kaggle/working/semeval2026-task12-dataset/dev_data/questions.jsonl
Docs file: /kaggle/working/semeval2026-task12-dataset/dev_data/docs.json


 ## 1.4 Import Repository and load dataset

In [11]:

!git clone https://github.com/irenebartolini02/LLM-abductive-event-reasoning.git
%cd LLM-abductive-event-reasoning

Cloning into 'LLM-abductive-event-reasoning'...
remote: Enumerating objects: 224, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 224 (delta 6), reused 20 (delta 4), pack-reused 200 (from 1)
Receiving objects: 100% (224/224), 31.02 MiB | 21.32 MiB/s, done.
Resolving deltas: 100% (106/106), done.
/kaggle/working/LLM-abductive-event-reasoning/LLM-abductive-event-reasoning


In [15]:
from utils.data_loader import load_jsonl, load_json, index_docs_by_topic

questions = load_jsonl(QUESTIONS_FILE)
docs = load_json(DOCS_FILE)
docs_by_topic = index_docs_by_topic(docs)

print("Num questions:", len(questions))
print("Num doc groups:", len(docs_by_topic))
print("Example question keys:", questions[0].keys())

Num questions: 400
Num doc groups: 36
Example question keys: dict_keys(['topic_id', 'id', 'target_event', 'option_A', 'option_B', 'option_C', 'option_D', 'golden_answer'])


## 1.5 Set Wandb Key

In [ ]:
from kaggle_secrets import UserSecretsClient

# Inizializza il client
user_secrets = UserSecretsClient()

# Leggi la tua API key W&B
api_key = user_secrets.get_secret("WANDB_API_KEY")

print("WANDB_API_KEY trovata:", api_key is not None)

# Imposta per wandb
import os
os.environ["WANDB_API_KEY"] = api_key

## 1.6. Loading Model



In [16]:

from utils.model_utils import load_model

# set yor model name
MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct"


model, tokenizer= load_model(MODEL_NAME)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

Model loaded on: cuda:0


## 1.7 Checkpoints

In [42]:
## Be Careful if you are in Colab Or Kaggle
CHECKPOINT_FILE = "/kaggle/working/checkpoint/evaluation_checkpoint.jsonl"

In [43]:
import os

checkpoint_dir = os.path.dirname(CHECKPOINT_FILE)

os.makedirs(checkpoint_dir, exist_ok=True)

print(f"Checkpoint folder ready at: {checkpoint_dir}")

Checkpoint folder ready at: /kaggle/working/checkpoint


In [44]:
import os 
import json 

results = []
total_score = 0

processed_uuids = set() # Set to check which question to skip (already processed)

if os.path.exists(CHECKPOINT_FILE):
    print(f"Found checkpoint: {CHECKPOINT_FILE}. Loading checkpoints...")
    with open(CHECKPOINT_FILE, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                try:
                    data = json.loads(line)
                    results.append(data)
                    processed_uuids.add(data['uuid']) # Memorize processed ID
                    total_score += data['score']
                except json.JSONDecodeError:
                    continue
    print(f"Restored {len(results)} previous results.")
    print(f"Restored results have avg score of {total_score/len(results)}")
else:
    print("No checkpoint found. Start a new evaluation")
    

No checkpoint found. Start a new evaluation


# **2. Experiments**

## 2.1 main model

In [ ]:
from sentence_transformers import CrossEncoder

embed_model = SentenceTransformer('BAAI/bge-base-en-v1.5')
reranker_model_name='cross-encoder/ms-marco-MiniLM-L-6-v2'
reranker = CrossEncoder(reranker_model_name, device=device)

In [ ]:
ENABLE_WANDB= False
ENABLE_DEBUG= True
MAX_SEARCH_STEPS=3

In [ ]:
import gc   # garbage collector
from utils.output_utils import clean_response, calculate_score, print_metrics
import json
import langchain.globals
import os
import pickle
from rag.RAGChain import RagChain, build_context, search_query
from agents.searcher import search_agent
from agents.reasoner import reasoner_agent
from agents.extractor import extractor_agent
import wandb


total_score = 0
count = 0
errors = 0


running_stats = {
    "correct": 0,
    "partial": 0,
    "wrong": 0
}

if ENABLE_WANDB:
    run_name = f"exp_{time.strftime('%Y%m%d_%H%M%S')}"
    
    run = wandb.init(
        project="LLM Reasoning Abductive",  # The project name on your dashboard
        name= run_name,        # Name of this specific run
        config={
            "k_per_option": 20,
            "k_final": 2,
            "chunk_size": 800,
            "chunk_overlap": 256,
        }
    )

    rag = RagChain(
        embed_model,
        reranker = reranker,
        k_per_option = wandb.config.k_per_option,
        k_final = wandb.config.k_final,
        chunk_size = wandb.config.chunk_size,
        chunk_overlap = wandb.config.chunk_overlap)
    rag.index_documents(docs_by_topic)


    # --- RESTORE FROM CHECKPOINT FOR WANDB LOGGING ---
    if os.path.exists(CHECKPOINT_FILE):
        print(f"Recupero checkpoint da {CHECKPOINT_FILE}...")
        with open(CHECKPOINT_FILE, 'r', encoding='utf-8') as f:
            for line in f:
                try:
                    data = json.loads(line.strip())
                    
                    # Aggiorniamo le variabili di memoria
                    results.append(data)
                    processed_uuids.add(data['uuid'])
                    
                    # Aggiorniamo le statistiche
                    score = data['score']
                    total_score += score
                    count += 1
                    
                    if score == 1.0:
                        running_stats["correct"] += 1
                    elif score > 0:
                        running_stats["partial"] += 1
                    else:
                        running_stats["wrong"] += 1
                    
                    # This reconstructs the graph for the historical data
                    if ENABLE_WANDB:
                        wandb.log({
                            "progress": count,
                            "global/correct": running_stats["correct"] / count,
                            "global/partial": running_stats["partial"] / count,
                            "global/error_rate": running_stats["wrong"] / count,
                            "global/avg_score": total_score / count,
                        })
                    
                except json.JSONDecodeError:
                    continue
        print(f"Ripristinati {count} elementi. Riprendo l'elaborazione...")
    else:
        print("Nessun checkpoint trovato. Inizio da zero.")
'''
# --- IMPORTANTE: PRENDO SOLO GLI ERRORI --- 
target_ids = set(error_idx) 
# 2. Filter the main list
# Replace 'uuid' with 'id' if that is your specific key
questions_to_retry = [q for q in questions if q.get('id') in target_ids]'''


print(f"Starting evaluation on {len(questions)} questions...")
search_list = []

# we use tqdm to see the progress bar
for entry in tqdm(questions[:200]):
    question_uuid = entry['id']
    
    
    
    # --- SKIP LOGIC ---

    # If UUID is in the processed sed, we skip the iteration
    if question_uuid in processed_uuids:
        continue

    # ------------------

    try:
        topic_id = entry['topic_id']
        question_uuid = entry['id']
        golden_ans = entry['golden_answer']
        target_event = entry['target_event']
        options = [entry["option_A"],entry["option_B"],entry["option_C"],entry["option_D"]]
        
        # Recover all documents for this topic
        context_docs = docs_by_topic.get(topic_id, [])

        # Create the initial context
        initial_context = build_context(rag,topic_id, target_event, options)

        #print(f"Initial Context:\n{initial_context}")

        search_iteration = 0
        is_sufficient = False
        total_queries = []
        while search_iteration < MAX_SEARCH_STEPS and not is_sufficient:    
            #print(f"============= Iteration {search_iteration} ===============")
            search_response_parsed = search_agent(initial_context, entry)
            #print(f"\nReasoning ================\n {search_response_parsed['reasoning']}")
            search_response_parsed["iteration"] = search_iteration
            
            is_sufficient = search_response_parsed['is_sufficient']
            #print(f"Is enough ?? -> {is_sufficient}")
            if not is_sufficient:
                queries = search_response_parsed['search_queries']
                #print(f"\n Ready to search for: {queries}")
                if queries:
                    for query in queries: 
                        if query in total_queries:
                            is_sufficient = True
                        else:
                            single_query_context = search_query(rag,query,topic_id, 2 + len(total_queries) )
                            #print(f"\n Retrieved Context for {query}:\n {single_query_context}")
                            total_queries.append(query)
                            initial_context += "\n\n" + single_query_context
                #print(f"\nContext after iteration {search_iteration}: \n{initial_context}")
                search_iteration += 1

        final_context = initial_context
        search_response_parsed ['context'] = final_context
        #print(f"The final context is \n{final_context}")

        causal_response_parsed = causal_agent(final_context,entry)

       # print(f"\n==== CAUSAL AGENT ==== ")
       # print(f"Reasoning: \n {causal_response_parsed['reasoning']}")
       # print(f"Answers:{causal_response_parsed['answer']}")
        reasoning = causal_response_parsed['reasoning']
        answer = causal_response_parsed['answer']

        pred_set = set(answer)
        score = calculate_score(pred_set,golden_ans)

        result_item = {
            "uuid": question_uuid,
            "topic_id": topic_id,
            "target_event": entry["target_event"],
            "options": options,
            "golden_raw": golden_ans,
            "prediction_set": list(pred_set),
            "reasoning_response_raw": reasoning,
            "score": score,
            "search_queries": total_queries,
            "total_iteration_quries": search_iteration,
            "context": final_context
        }

        if ENABLE_DEBUG:
            print("\n" + "="*100)
            print(f"DEBUG REPORT | UUID: {question_uuid}")
            print(f"TARGET EVENT: {entry['target_event']}")
            print("-" * 100)
            
            print("OPTIONS:")
            # Automatically maps 0->A, 1->B, 2->C, etc.
            for i, opt in enumerate(options):
                print(f"  [{chr(65+i)}]: {opt}")
        
            print("-" * 50)
            print(f"GOLDEN ANSWER   : {golden_ans}")
            print(f"PREDICTED SET   : {list(pred_set)}")
            print(f"SCORE           : {score}")
            
            print("-" * 50)
            print("QUERIES SEARCHED:")
            if total_queries:
                for i, q in enumerate(total_queries, 1):
                    print(f"  {i}. {q}")
            else:
                print("  (None)")
                
            print("="*100 + "\n")

        
        # --- IMMEDIATE SAVING (Checkpoint) ---
        # Append (mode 'a') a new line to the JSONL file
        # Basically, we save in a json (on the drive) every computed answer,
        # so that if something goes wrong we can restore it from the drive
        with open(CHECKPOINT_FILE, 'a', encoding='utf-8') as f:
            f.write(json.dumps(result_item) + "\n")

        
        # Updating variables in memory
        results.append(result_item)
        processed_uuids.add(question_uuid)
        total_score += score
        count += 1

        if score == 1.0:
            running_stats["correct"] += 1
        elif score > 0:
            running_stats["partial"] += 1
        else:
            running_stats["wrong"] += 1

        if ENABLE_WANDB:
            wandb.log({
                "progress": count,
                "global/correct": running_stats["correct"] / count,
                "global/partial": running_stats["partial"] / count,
                "global/error_rate": running_stats["wrong"] / count,
                "global/avg_score": total_score / count,
            })        
        
    except RuntimeError as e:
        if "out of memory" in str(e):
            print(f"UUID:{entry['id']} process skipped due to OOM")
            # forcely free the cache
            del inputs
            torch.cuda.empty_cache()
            gc.collect()
            errors += 1
            continue
        else:
            print(f"Errore generico: {e}")
            continue

if ENABLE_WANDB:
    wandb.finish()

## 2.2 Extension - CausalRag evaluation

In [ ]:
!pip install langchain-experimental==0.0.64
!pip install langchain-huggingface==0.0.3
!pip install rank_bm25
!pip install json-repair
!pip install faiss-cpu

In [ ]:
import gc   # garbage collector
import time
import wandb
import re
from rag.CausalRAG import CausalRAG, build_causal_context, search_causal_query
from agents.searcher import search_agent
from agents.reasoner import reasoner_agent
from agents.extractor import extractor_agent


DATASET_FOLDER="/kaggle/input/causal-rag-datasets/causal_dataset"
total_score = 0
count = 0
errors = 0


running_stats = {
    "correct": 0,
    "partial": 0,
    "wrong": 0
}


if ENABLE_WANDB:
    run_name = f"exp_{time.strftime('%Y%m%d_%H%M%S')}"
    
    run = wandb.init(
        project="LLM Reasoning Abductive",  # The project name on your dashboard
        name= run_name,        # Name of this specific run
        config={
            "k_per_option": 20,
            "k_final": 2,
            "chunk_size": 800,
            "chunk_overlap": 256,
        }
    )


    # --- RESTORE FROM CHECKPOINT FOR WANDB LOGGING ---
    if os.path.exists(CHECKPOINT_FILE):
        print(f"Recupero checkpoint da {CHECKPOINT_FILE}...")
        with open(CHECKPOINT_FILE, 'r', encoding='utf-8') as f:
            for line in f:
                try:
                    data = json.loads(line.strip())
                    
                    # Aggiorniamo le variabili di memoria
                    results.append(data)
                    processed_uuids.add(data['uuid'])
                    
                    # Aggiorniamo le statistiche
                    score = data['score']
                    total_score += score
                    count += 1
                    
                    if score == 1.0:
                        running_stats["correct"] += 1
                    elif score > 0:
                        running_stats["partial"] += 1
                    else:
                        running_stats["wrong"] += 1
                    
                    # This reconstructs the graph for the historical data
                    if ENABLE_WANDB:
                        wandb.log({
                            "progress": count,
                            "global/correct": running_stats["correct"] / count,
                            "global/partial": running_stats["partial"] / count,
                            "global/error_rate": running_stats["wrong"] / count,
                            "global/avg_score": total_score / count,
                        })
                    
                except json.JSONDecodeError:
                    continue
        print(f"Ripristinati {count} elementi. Riprendo l'elaborazione...")
    else:
        print("Nessun checkpoint trovato. Inizio da zero.")
'''
# --- IMPORTANTE: PRENDO SOLO GLI ERRORI --- 
target_ids = set(error_idx) 
# 2. Filter the main list
# Replace 'uuid' with 'id' if that is your specific key
questions_to_retry = [q for q in questions if q.get('id') in target_ids]'''



search_list = []

# we use tqdm to see the progress bar
questions_id = [q for q in questions if q.get('topic_id')<=20]
total=len(questions_id)
print(f"Starting evaluation on {total} questions...")

for i, entry in enumerate(tqdm(questions_id)):
    question_uuid = entry['id']
    
    # --- SKIP LOGIC ---

    # If UUID is in the processed sed, we skip the iteration
    if question_uuid in processed_uuids:
        continue

    # ------------------

    try:
        topic_id = entry['topic_id']
        question_uuid = entry['id']
        golden_ans = entry['golden_answer']
        target_event = entry['target_event']
        options = [entry["option_A"],entry["option_B"],entry["option_C"],entry["option_D"]]
        
        # Recover all documents for this topic
        context_docs = docs_by_topic.get(topic_id, [])

        # Create the initial context CAUSAL RAG
        rag= CausalRAG(model, tokenizer, 3,3)
        rag.load_causal_rag(topic_id, DATASET_FOLDER)
        initial_context =  build_causal_context(rag, target_event, options)

        print(f"Initial Context:\n{initial_context}")

        search_iteration = 0
        is_sufficient = False
        total_queries = []
        answer=[]
        
        while search_iteration < MAX_SEARCH_STEPS and not answer:    
            #print(f"============= Iteration {search_iteration} ===============")
            search_response_parsed = search_agent(model, tokenizer, initial_context, entry)
            print(f"\nReasoning ================\n {search_response_parsed['reasoning']}")
            search_response_parsed["iteration"] = search_iteration
            
            is_sufficient = search_response_parsed['is_sufficient']
            
            if not is_sufficient:
                queries = search_response_parsed['queries']
                print(f"\n Ready to search for: {queries}")
                if queries:
                    for query in queries: 
                        if query in total_queries:
                            is_sufficient = True
                        else:
                             single_query_context = search_causal_query(rag, query, target_event, options) 
                             total_queries.append(query)
                             if single_query_context not in initial_context:
                                initial_context += "\n\n" + single_query_context
                
            search_iteration += 1

            final_context = initial_context
            search_response_parsed ['context'] = final_context
            if total_queries:
                print(f"The final context is \n{final_context}")
            causal_response_parsed = reasoner_agent(model, tokenizer, final_context,entry)
            answer= extractor_agent(model, tokenizer, causal_response_parsed['reasoning'])

        print(f"\n==== CAUSAL AGENT ==== ")
        print(f"Reasoning: \n {causal_response_parsed['reasoning']}")
        print(f"Answers:{answer}")
        reasoning = causal_response_parsed['reasoning']

        pred_set = set(answer)
        score = calculate_score(pred_set,golden_ans)

        result_item = {
            "uuid": question_uuid,
            "topic_id": topic_id,
            "target_event": entry["target_event"],
            "options": options,
            "golden_raw": golden_ans,
            "prediction_set": list(pred_set),
            "reasoning_response_raw": reasoning,
            "score": score,
            "search_queries": total_queries,
            "total_iteration_quries": search_iteration,
            "context": final_context
        }


        if ENABLE_DEBUG:
            print("\n" + "="*100)
            print(f"DEBUG REPORT | UUID: {question_uuid}")
            print(f"TARGET EVENT: {entry['target_event']}")
            print("-" * 100)
            
            print("OPTIONS:")
            # Automatically maps 0->A, 1->B, 2->C, etc.
            for i, opt in enumerate(options):
                print(f"  [{chr(65+i)}]: {opt}")
        
            print("-" * 50)
            print(f"GOLDEN ANSWER   : {golden_ans}")
            print(f"PREDICTED SET   : {list(pred_set)}")
            print(f"SCORE           : {score}")
            
            print("-" * 50)
            print("QUERIES SEARCHED:")
            if total_queries:
                for i, q in enumerate(total_queries, 1):
                    print(f"  {i}. {q}")
            else:
                print("  (None)")
                
            print("="*100 + "\n")

        
        # --- IMMEDIATE SAVING (Checkpoint) ---
        # Append (mode 'a') a new line to the JSONL file
        # Basically, we save in a json (on the drive) every computed answer,
        # so that if something goes wrong we can restore it from the drive
        with open(CHECKPOINT_FILE, 'a', encoding='utf-8') as f:
            f.write(json.dumps(result_item) + "\n")

        
        # Updating variables in memory
        results.append(result_item)
        processed_uuids.add(question_uuid)
        total_score += score
        count += 1

        if score == 1.0:
            running_stats["correct"] += 1
        elif score > 0:
            running_stats["partial"] += 1
        else:
            running_stats["wrong"] += 1

        if ENABLE_WANDB:
            wandb.log({
                "progress": count,
                "global/correct": running_stats["correct"] / count,
                "global/partial": running_stats["partial"] / count,
                "global/error_rate": running_stats["wrong"] / count,
                "global/avg_score": total_score / count,
            })        
        
    except RuntimeError as e:
        if "out of memory" in str(e):
            print(f"UUID:{entry['id']} process skipped due to OOM")
            # forcely free the cache
            del inputs
            torch.cuda.empty_cache()
            gc.collect()
            errors += 1
            continue
        else:
            print(f"Errore generico: {e}")
            continue

if ENABLE_WANDB:
    wandb.finish()

Starting evaluation on 188 questions...


  0%|          | 0/188 [00:00<?, ?it/s]Device set to use cuda:0


✓ Oggetto Ricostruito: 413 nodi. BM25Retriever pronto.
Initial Context:
The Capitol building was breached by protesters. Protesters engaged in physical conflict with law enforcement officers at the Capitol. President-elect Trump encouraged his supporters to march to the Capitol during a rally. Protesters stormed the U.S. Capitol.

Reasoning ================
 Option C ("Supporters stormed the U.S. Capitol") is a plausible cause for the event since it directly relates to the breach of the Capitol building. Options A, B, and D are not directly related to the event of Twitter removing Trump's tweets and locking his account.

 Ready to search for: ['Trump\\', ',']
The final context is 
The Capitol building was breached by protesters. Protesters engaged in physical conflict with law enforcement officers at the Capitol. President-elect Trump encouraged his supporters to march to the Capitol during a rally. Protesters stormed the U.S. Capitol.

Violent uprising against the government --[CAUSES

  1%|          | 1/188 [01:21<4:14:58, 81.81s/it]Device set to use cuda:0



==== CAUSAL AGENT ==== 
Reasoning: 
 Event Type: Decisional
Event Analysis: The event describes a decision made by Twitter to remove Trump's tweets and lock his account. This decision is likely influenced by the actions taken by supporters storming the Capitol.

### Analysis of Each Option

#### Option A: "A woman was shot inside the Capitol and died."
**Temporal Check:** FAIL.
- The shooting and death occurred during the breach, but the event mentions that Twitter took action after the storming of the Capitol.
**Mechanism:** The shooting and death are a consequence of the breach, not a direct cause of Twitter's decision.
**Counterfactual:** If the shooting and death did not occur, Twitter might still have taken action against Trump's tweets due to the broader context of the breach.
**Verdict:** REJECT - Spurious correlation.

#### Option B: "Pipe bombs were reported at the Republican National Committee building and the Democratic National Committee headquarters."
**Temporal Check:** 

  1%|          | 1/188 [02:07<6:38:00, 127.70s/it]


KeyboardInterrupt: 

In [47]:
from utils.output_utils import print_metrics
print_metrics(results, MODEL_NAME)

======= Qwen/Qwen2.5-7B-Instruct Causal Reasoning Results =======
Total questions: 1
Correct answers: 1 - 100.0%
Partial answers: 0 - 0.0%
Wrong answers: 0 - 0.0%
Total score: 1.0
Performance of the score: 100.0%
